# HUGGENFACE BERT MODEL EĞİTİM



In [ ]:
# Öncelikle Hugging Face'in transformers kütüphanesini yükleyin
!pip install transformers


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Modeli ve tokenizörü yükleyin
model_name = "anilguven/distilbert_tr_turkish_hotel_reviews"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Test etmek istediğiniz bir cümle yazın
sentence = "mal"

# Tokenleştirme ve modelden tahmin yapma
inputs = tokenizer(sentence, return_tensors='pt')
outputs = model(**inputs)

# Logitleri alıp yumuşatılmış sınıf olasılıklarını hesaplayın
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

# En yüksek olasılığa sahip olan sınıfı alın
predicted_class = probabilities.argmax().item()

custom_labels = {
    0: "negatif",
    1: "pozitif"
}

# Etiketi kullanıcı dostu bir formatta yazdırın
predicted_label = custom_labels[predicted_class]
print("Tahmin Edilen Sınıf:", predicted_label)

# Sonuçları yazdırın
print("Olasılıklar:", probabilities.detach().numpy())


Tahmin Edilen Sınıf: negatif
Olasılıklar: [[0.98069304 0.01930696]]


In [ ]:
# Modelin sınıf etiketlerini alalım
labels = model.config.id2label
print("Sınıf Etiketleri:", labels)


Sınıf Etiketleri: {0: 'LABEL_0', 1: 'LABEL_1'}


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Modeli ve tokenlayıcıyı yükleyin
model_name = "anilguven/distilbert_tr_turkish_hotel_reviews"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sınıf etiketlerini değiştirme
custom_labels = {
    0: "negatif",
    1: "pozitif"
}

# Test etmek istediğiniz bir cümle yazın
sentence = "köpek bile burda bağlasan durmaz"

# Tokenize etme ve modelden tahmin yapma
inputs = tokenizer(sentence, return_tensors='pt')
outputs = model(**inputs)

# Logitleri alıp olasılıkları hesaplayın
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

# En yüksek olasılığa sahip olan sınıfı alın
predicted_class = probabilities.argmax().item()

# Etiketi kullanıcı dostu bir formatta yazdırın
predicted_label = custom_labels[predicted_class]
print("Tahmin Edilen Sınıf:", predicted_label)

# Olasılıkları ve sınıfı yazdırın
print("Olasılıklar:", probabilities.detach().numpy())


Tahmin Edilen Sınıf: negatif
Olasılıklar: [[0.5003147  0.49968532]]


#TRAİN OWN DATA

In [ ]:
!pip install transformers torch datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/AISentiment/TrainData/yeni_veri_seti.csv')
df.head()

,yorum,labels
0,yatağı çekip çıkarmak rahatsızdı şiltenin yayl...,1
1,ücretsiz wi-fi yalnızca resepsiyonda mevcut ka...,0
2,mükemmel tedavi rakipsiz tesisler iyi düşünülm...,1
3,kaba gece müdürü gece kulübü fedaisi davranıyo...,1
4,wesley swales otel müdürü lobideki balık kokus...,1


In [ ]:
#from transformers import AutoTokenizer

# Tokenizer'ı önceden eğitilmiş bir modelden yükleme
#tokenizer = AutoTokenizer.from_pretrained('anilguven/distilbert_tr_turkish_hotel_reviews')
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('anilguven/distilbert_tr_turkish_hotel_reviews',force_download=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/343 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/343 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

In [ ]:
def process_data(row):

    text = row['yorum']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    label = 0
    if row['labels'] == 'positive':
        label += 1

    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [ ]:
processed_data = []

for i in range(len(df[:1000])):
    processed_data.append(process_data(df.iloc[i]))

In [ ]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [ ]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [ ]:
#from transformers import AutoModelForSequenceClassification

#model = AutoModelForSequenceClassification.from_pretrained(
 #   'dbmdz/distilbert-base-turkish-cased',
  #  num_labels=2
#)

from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    'anilguven/distilbert_tr_turkish_hotel_reviews',
    num_labels=2,force_download=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/272M [00:00<?, ?B/s]

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_name = "anilguven/distilbert_tr_turkish_hotel_reviews"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Eğitim epoch sayısı
    per_device_train_batch_size=16,  # Eğitimdeki batch boyutu
    per_device_eval_batch_size=16,  # Değerlendirmedeki batch boyutu
    warmup_steps=500,  # Öğrenme oranı için ısınma adımları
    weight_decay=0.01,  # Ağırlık bozulması
    evaluation_strategy="epoch",  # Değerlendirme stratejisi
    save_strategy="epoch",  # Kaydetme stratejisi
    logging_dir='./logs',  # Log kayıtları için dizin
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
)




In [ ]:
trainer.train()  # Eğitimi başlatın

Epoch,Training Loss,Validation Loss
1,No log,1.058012
2,No log,0.000138
3,No log,0.000041


TrainOutput(global_step=150, training_loss=0.9399928792317709, metrics={'train_runtime': 32.1791, 'train_samples_per_second': 74.582, 'train_steps_per_second': 4.661, 'total_flos': 79480439193600.0, 'train_loss': 0.9399928792317709, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 4.12371737184003e-05,
 'eval_runtime': 0.7531,
 'eval_samples_per_second': 265.576,
 'eval_steps_per_second': 17.262,
 'epoch': 3.0}

In [ ]:
#model.save_pretrained('G:/model/')
# Save the model
model_path = '/content/drive/MyDrive/AISentiment/Bert Modeli/my_custom_model'  # Adjust as needed
model.save_pretrained(model_path)

# Save the tokenizer
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/AISentiment/Bert Modeli/my_custom_model/tokenizer_config.json',
 '/content/drive/MyDrive/AISentiment/Bert Modeli/my_custom_model/special_tokens_map.json',
 '/content/drive/MyDrive/AISentiment/Bert Modeli/my_custom_model/vocab.txt',
 '/content/drive/MyDrive/AISentiment/Bert Modeli/my_custom_model/added_tokens.json')

# Model Testing

In [ ]:
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizer
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path= '/content/drive/MyDrive/AISentiment/Bert Modeli/my_custom_model'
new_model = DistilBertForSequenceClassification.from_pretrained(path).to(device)
new_tokenizer = DistilBertTokenizer.from_pretrained(path)


In [ ]:
def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    outputs = new_model(**encoding)

    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze())
    probs = probs.detach().cpu().numpy()

    label = int(probs[1] > probs[0])

    sentiment = "Positive" if label == 1 else "Negative"
    probability = probs[1] if label == 1 else probs[0]

    return {
        'sentiment': sentiment,
        'probability': probability
    }




In [ ]:
result = get_prediction("harika bir yer")
print("Sentiment:", result['sentiment'], "Probability:", result['probability'])


Sentiment: Positive Probability: 0.95409703


In [ ]:
#LABEL_0: olumlu inceleme
#LABEL_1: olumsuz yorum